In [1]:
import pickle
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from time import time

In [2]:
preproces = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [3]:
bert = hub.load("https://tfhub.dev/google/electra_large/2")


In [4]:
def embedd_data(data, step = 200, end = None):
    #step = 50 is tuned for rtx 2060!
    #step is GPU (and Vram) dependant!
    s = time()
    ba = []
    misses = []
    for i in range(0,data.shape[0],step):
        print(i)
        if end:
            if i >= end: return ba
        try:
            inp = preproces(data[i:i+step])
            outp = bert(inp, training=False)
            ba.append(outp['pooled_output'])
            tf.keras.backend.clear_session()
        except:
            misses.append(i)
            print(i)
    print('for loop is over')
    print(abs(time()-s))
    return ba

In [5]:
baseline = pd.read_csv('CSVs/hate_spe.csv')
c0 = baseline[baseline['class'] == 2]['tweet'].values
c1 = pd.read_csv('CSVs/incels.csv')['text'].values

In [6]:
baseline = embedd_data(c0, step = 50)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
for loop is over
191.10306072235107


In [14]:
print('Saving baseline')
with open('encoded_baseline.pkl', 'wb') as outp:
    pickle.dump(baseline, outp, pickle.HIGHEST_PROTOCOL)

Saving baseline


In [8]:
problem = embedd_data(c1, step = 50, end = 8000)

0
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000


In [15]:
print('Saving cl1')
with open('encoded_c1_incels.pkl', 'wb') as outp:
    pickle.dump(problem, outp, pickle.HIGHEST_PROTOCOL)

Saving cl1


In [16]:
with open('encoded_baseline.pkl', 'rb') as outp:
    b1 = pickle.load(outp)

with open('encoded_c1_incels.pkl', 'rb') as outp:
    c1 = pickle.load(outp)

print(len(c1), len(b1))


159 84


In [17]:
len(b1)

84

In [18]:
84*50

4200

In [19]:
len(problem)

159